In [1]:
import numpy as np
import pandas as pd

In [21]:
import requests

In [4]:
pip install BeautifulSoup4

Note: you may need to restart the kernel to use updated packages.


In [5]:
from bs4 import BeautifulSoup

In [6]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(data, 'html.parser')

In [7]:
postalCodeList = []
boroughList = []
neighborhoodList = []

for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n'))

In [8]:
toronto = [('PostalCode', postalCodeList),
                      ('Borough', boroughList),
                      ('Neighborhood', neighborhoodList)]
df = pd.DataFrame.from_dict(dict(toronto))
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [9]:
df= df[df.Borough != "Not assigned"].reset_index(drop=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [10]:
df = df.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [11]:
for index, row in df.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
df.tail()

,PostalCode,Borough,Neighborhood
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."
102,M9W,Etobicoke,Northwest


In [12]:
gdf=pd.read_csv("http://cocl.us/Geospatial_data")
gdf.rename(columns={"Postal Code": "PostalCode", "Latitude": "Latitude", "Longitude":"Longitude"})
gdf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
df2 = df.set_index('PostalCode')
gdf_temp = gdf.set_index('Postal Code')
df4 = pd.concat([df2, gdf_temp], axis=1, join='inner')

df4.index.name = 'PostalCode'
df4.reset_index(inplace=True)

df4.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [14]:
df4.shape

(103, 5)

In [16]:
toronto_data = df[df['Borough'] == 'Scarborough'].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [17]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import matplotlib.pyplot as plt #library to plot a graph
from sklearn.cluster import KMeans #library for K Means
%matplotlib inline

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library for map

print('Folium installed')
print('Libraries imported.')

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\varun\Anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.8.0                |           py37_1         3.0 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.20.0-py_0

The following packages will be UPDATED:

  conda                      pkgs/main::conda-4.7.10-py37_0 --> conda-forge::conda-4.8.0-py37_1




conda-4.8.0          | 3.0 M



==> WARNING: A newer version of conda exists. <==
  current version: 4.7.10
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda




Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/win-64::anaconda==2019.07=py37_0
  - defaults/win-64::numba==0.44.1=py37hf9181ef_0
done

## Package Plan ##

  environment location: C:\Users\varun\Anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _anaconda_depends-2019.03  |           py37_0           6 KB
    altair-4.0.0               |             py_0         606 KB  conda-forge
    anaconda-custom            |           py37_1           3 KB
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         182 KB  conda-forge
    certifi

In [19]:
CLIENT_ID = 'NAEG5PK5YPSB5KCU05ERESVK3WMVILXOY3WXLGLCZJNRGAXF'
CLIENT_SECRET = 'CL0BLCB3S21FUPV0MKZJMPSN1VOI4IXKFGGCWV2BQ4ECGKDV'
VERSION = '20191218'
LIMIT = 50

In [22]:
address = 'Scarborough, ON, Canada'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.773077 -79.257774


In [24]:
search_query = 'Restaurant'
Radius = 10000
print(search_query + ' .... OK!')

Restaurant .... OK!


In [25]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, Radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=NAEG5PK5YPSB5KCU05ERESVK3WMVILXOY3WXLGLCZJNRGAXF&client_secret=CL0BLCB3S21FUPV0MKZJMPSN1VOI4IXKFGGCWV2BQ4ECGKDV&ll=43.773077,-79.257774&v=20191218&query=Restaurant&radius=10000&limit=50'

In [26]:
results = requests.get(url).json()

In [28]:
# assigning relevant part of JSON to venues
venues = results['response']['venues']

# tranforming venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",False,4b29e021f964a520d4a324e3,4386 Sheppard Ave. E,CA,Toronto,Canada,at Brimley Rd.,1920,"[4386 Sheppard Ave. E (at Brimley Rd.), Toront...","[{'label': 'display', 'lat': 43.78777423062292...",43.787774,-79.270294,NaN,M1S 1T8,ON,Perfect Chinese Restaurant 雅瓊海鮮酒家,v-1576726611,NaN
1,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",False,4be303c52fc7d13ae879083a,4271 Sheppard Ave. E,CA,Scarborough,Canada,btwn Brimley & Midland Ave.,2050,[4271 Sheppard Ave. E (btwn Brimley & Midland ...,"[{'label': 'display', 'lat': 43.78593747713555...",43.785937,-79.276031,NaN,M1S 4G4,ON,Beef Noodle Restaurant 老李牛肉麵,v-1576726611,NaN
2,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",False,4b79de5ff964a52036172fe3,4002 Sheppard Ave. E,CA,Scarborough,Canada,at Kennedy Rd.,2677,"[4002 Sheppard Ave. E (at Kennedy Rd.), Scarbo...","[{'label': 'display', 'lat': 43.78367044432938...",43.783670,-79.287679,NaN,M1S 4R5,ON,Very Fair Chinese Restaurant 同德樓,v-1576726611,NaN
3,"[{'id': '4bf58dd8d48988d113941735', 'name': 'K...",False,4d2f8a98789a8cfa6b0826c6,9 Glen Watford Dr.,CA,Scarborough,Canada,at Sheppard Ave. E,2072,"[9 Glen Watford Dr. (at Sheppard Ave. E), Scar...","[{'label': 'display', 'lat': 43.78646767038441...",43.786468,-79.275693,NaN,M1S 2B9,ON,In Cheon House Korean & Japanese Restaurant 인천관,v-1576726611,NaN
4,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",False,4bf96c435317a593a23a017f,1225 Kennedy Rd,CA,Toronto,Canada,at Forbes (Between Lawrence and Ellesmere),2410,[1225 Kennedy Rd (at Forbes (Between Lawrence ...,"[{'label': 'display', 'lat': 43.75604153945313...",43.756042,-79.276276,NaN,M1P 4Y1,ON,Karaikudi Chettinad South Indian Restaurant,v-1576726611,33638214


We will retrive the name of restaurants in Scarborough

In [35]:
restaurant_names_Scarborough=dataframe['name']
restaurant_names_Scarborough

0                     Perfect Chinese Restaurant 雅瓊海鮮酒家
1                          Beef Noodle Restaurant 老李牛肉麵
2                      Very Fair Chinese Restaurant 同德樓
3       In Cheon House Korean & Japanese Restaurant 인천관
4           Karaikudi Chettinad South Indian Restaurant
5             South Sea Fish Village Chinese Restaurant
6                            Sagano Japanese Restaurant
7                    The Royal Chinese Restaurant 避風塘小炒
8                   Old Neighbour Restaurant 老街坊天津韩记包子铺
9                   Best Friends Chinese Restaurant 會賓樓
10                     Makkalchon Korean Restaurant 맛깔촌
11                            Sammy's Family Restaurant
12                             Fortune House Restaurant
13                           Supreme Restaurant and Bar
14                              JXY Dumpling Restaurant
15                            Wok Wok Restaurant 恒記粥麵小廚
16                        Fortune Stone Restaurant 滾石餐廳
17                            Xin Jiang Restaura